<a href="https://colab.research.google.com/github/zahraDehghanian97/Apriori/blob/master/Apriori.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# load data

In [20]:
!wget https://github.com/zahraDehghanian97/Apriori/blob/master/records.txt

--2022-12-19 11:51:44--  https://github.com/zahraDehghanian97/Apriori/blob/master/records.txt
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘records.txt’

records.txt             [ <=>                ] 135.62K  --.-KB/s    in 0.02s   

2022-12-19 11:51:44 (6.11 MB/s) - ‘records.txt’ saved [138877]



In [21]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [22]:
import pandas as pd
items = []
extended = []
# with open("/content/gdrive/MyDrive/Colab Notebooks/records.txt") as file:
with open("records.txt") as file:
    for line in file:
        s = line.split(" ")
        items.append(s)
        extended.extend(s)
print(len(items))
unique_items = list(set(extended))
print(len(unique_items))

1543
2836


<h2>Convert Items to Number</h2>

In [23]:
dictionaries =dict(zip(tuple(unique_items), tuple(range(len(unique_items)))))
numbered_items = []
for i in range(len(items)):
  temp = []
  for j in items[i] :
    temp.append(dictionaries[j])
  numbered_items.append(list(set(temp)))

print(numbered_items[:10])


[[232], [232], [232], [232], [232], [232], [232], [717, 1854], [2116, 397, 1682, 1974, 2716, 1596], [0, 1066]]


<h2>Change Categorical data into numbered</h2>

In [24]:
dict_data = {'items': numbered_items}
df = pd.DataFrame.from_dict(dict_data)
items = numbered_items
df

,items
0,[232]
1,[232]
2,[232]
3,[232]
4,[232]
...,...
1538,[232]
1539,"[0, 1536, 1742, 1145, 317, 1597]"
1540,"[0, 2578]"
1541,[948]


<h2>First Frequent Itemset (L1)</h2>

In [25]:
all_values = df['items'].values
from collections import Counter
MyList = []
for i in all_values:
  MyList.extend(list(set(i)))
c = Counter(MyList)
count_candidate1_df = pd.DataFrame.from_records(c.most_common(), columns=['itemset','sup'])
temp = []
for i in count_candidate1_df["itemset"]:
  temp.append([i])
count_candidate1_df["itemset"] = temp
count_candidate1_df


,itemset,sup
0,[0],1003
1,[232],427
2,[1614],127
3,[1536],113
4,[2629],105
...,...,...
2831,[1742],1
2832,[1145],1
2833,[317],1
2834,[2578],1


In [39]:
def filter_sup(candidate):
    minimum_sup = 50
    filtering = candidate['sup'] > minimum_sup
    freq = candidate[filtering]
    return freq

In [40]:
freq_itemset1 = filter_sup(count_candidate1_df)
freq_itemset1_reset = freq_itemset1.reset_index(drop=True)
freq_itemset1=freq_itemset1_reset
freq_itemset1

,itemset,sup
0,[0],1003
1,[232],427
2,[1614],127
3,[1536],113
4,[2629],105
5,[1880],95
6,[1699],68
7,[2059],63
8,[1669],63
9,[131],62


# Second Frequent Itemset (L2)


In [41]:
import numpy
def join(itemset1,itemset2):
    join_candidate = []
    list_join_candidate = []
    for i in range(len(itemset1['itemset'])):
        for j in range(len(itemset2['itemset'])):
            itemset_i = set(itemset1['itemset'][i])
            itemset_j = set(itemset2['itemset'][j])
            union_candidate = itemset_i.union(itemset_j)
            if ((len(union_candidate)== (len(itemset_i)+len(itemset_j))) and (union_candidate not in join_candidate)):
                join_candidate.append(set(union_candidate))
                list_join_candidate.append(list(union_candidate))
    
    return list_join_candidate

In [42]:
candidate2_list = join(freq_itemset1,freq_itemset1)
len(candidate2_list)

210

In [43]:
def count_support(database_dataframe, candidate_list):
    count_candidate = []
    for i in range(len(candidate_list)):
        count_candidate.append((candidate_list[i], 0))
    df_candidate = pd.DataFrame(count_candidate, columns=['itemset', 'sup'])
    for i in range(len(database_dataframe)):
        for j in range(len(count_candidate)):
          if (set(df_candidate['itemset'][j])).issubset(set(database_dataframe['items'][i])): 
            df_candidate.loc[j, 'sup'] += 1
    return df_candidate

In [44]:
count_candidate2_df = count_support(df, candidate2_list)
freq_itemset2 = filter_sup(count_candidate2_df)
freq_itemset2 = freq_itemset2.reset_index(drop=True)
freq_itemset2

,itemset,sup
0,"[0, 232]",92
1,"[0, 1614]",127
2,"[0, 1536]",107
3,"[0, 2629]",104
4,"[0, 1880]",84
...,...,...
60,"[540, 2117]",51
61,"[540, 647]",51
62,"[1393, 2117]",51
63,"[1393, 647]",51


<h2>Third Frequent Itemset (L3)</h2>

In [45]:
join_result = join(freq_itemset2,freq_itemset1)
print('join result : ')
print(join_result)

join result : 
[[0, 232, 1614], [0, 232, 1536], [0, 232, 2629], [0, 232, 1880], [0, 232, 1699], [0, 232, 2059], [0, 232, 1669], [0, 232, 131], [0, 232, 180], [0, 232, 937], [0, 232, 729], [0, 232, 540], [0, 232, 2096], [0, 232, 2381], [0, 232, 2660], [0, 232, 2058], [0, 232, 1393], [0, 232, 2117], [0, 232, 647], [0, 1536, 1614], [0, 2629, 1614], [0, 1880, 1614], [0, 1699, 1614], [0, 2059, 1614], [0, 1669, 1614], [0, 131, 1614], [0, 180, 1614], [0, 937, 1614], [0, 729, 1614], [0, 540, 1614], [0, 2096, 1614], [0, 2381, 1614], [0, 2660, 1614], [0, 2058, 1614], [0, 1393, 1614], [0, 2117, 1614], [0, 1614, 647], [0, 1536, 2629], [0, 1536, 1880], [0, 1536, 1699], [0, 1536, 2059], [0, 1536, 1669], [0, 1536, 131], [0, 1536, 180], [0, 1536, 937], [0, 1536, 729], [0, 1536, 540], [0, 1536, 2096], [0, 1536, 2381], [0, 1536, 2660], [0, 1536, 2058], [0, 1536, 1393], [0, 1536, 2117], [0, 1536, 647], [0, 1880, 2629], [0, 1699, 2629], [0, 2059, 2629], [0, 2629, 1669], [0, 131, 2629], [0, 180, 2629], [0,

## Pruning

In [46]:
def get_subsets(candidate):
    temp = []
    final = []
    for i in range(len(candidate)):
        for j in range(len(candidate)):
            if i != j:
                temp.append(candidate[j])
        temp_set = set(temp)
        final.append(temp_set)
        temp.clear()
    # print('Subset from {} : {}'.format(candidate, final))
    return final

def pruning(candidate_set, freq_itemset):
    
    temp = []
    for idx, value in enumerate(candidate_set):
        print("---------------------------------------------------")
        print('Candidate to evaluate : ', value)
        list_candidate = list(value)
        temp_candidates = (get_subsets(list_candidate))
        checks = True
        for temp_item in temp_candidates:
            c = False
            for i in freq_itemset['itemset'] :
              if temp_item == set(i):
                c = True
            print('\nCheck {} candidate in Previous Frequent Itemset result : {} '.format(temp_item , c))
            checks = checks and c    
            if not c :
              break
        if checks : 
            print("++++++++++++++++++++++++++++++++++++++++++++++++++++++")
            print('\nAll of {} subset contained in \n{}'.format(candidate_set, freq_itemset))
            if value not in temp:
                temp.append(value)
    return temp

In [47]:
candidate3_list = pruning(join_result, freq_itemset2)
print("final candidate 3 list : ")
candidate3_list

Streaming output truncated to the last 5000 lines.

Check {2629, 1614} candidate in Previous Frequent Itemset result : True 

Check {2058, 1614} candidate in Previous Frequent Itemset result : False 
---------------------------------------------------
Candidate to evaluate :  [1393, 2629, 1614]

Check {2629, 1614} candidate in Previous Frequent Itemset result : True 

Check {1393, 1614} candidate in Previous Frequent Itemset result : True 

Check {1393, 2629} candidate in Previous Frequent Itemset result : True 
++++++++++++++++++++++++++++++++++++++++++++++++++++++

All of [[0, 232, 1614], [0, 232, 1536], [0, 232, 2629], [0, 232, 1880], [0, 232, 1699], [0, 232, 2059], [0, 232, 1669], [0, 232, 131], [0, 232, 180], [0, 232, 937], [0, 232, 729], [0, 232, 540], [0, 232, 2096], [0, 232, 2381], [0, 232, 2660], [0, 232, 2058], [0, 232, 1393], [0, 232, 2117], [0, 232, 647], [0, 1536, 1614], [0, 2629, 1614], [0, 1880, 1614], [0, 1699, 1614], [0, 2059, 1614], [0, 1669, 1614], [0, 131, 1614], [0

[[0, 2629, 1614],
 [0, 2059, 1614],
 [0, 1669, 1614],
 [0, 180, 1614],
 [0, 937, 1614],
 [0, 729, 1614],
 [0, 540, 1614],
 [0, 2660, 1614],
 [0, 1393, 1614],
 [0, 2117, 1614],
 [0, 1614, 647],
 [0, 2059, 2629],
 [0, 180, 2629],
 [0, 937, 2629],
 [0, 729, 2629],
 [0, 540, 2629],
 [0, 1393, 2629],
 [0, 2117, 2629],
 [0, 2629, 647],
 [0, 2059, 180],
 [0, 937, 2059],
 [0, 729, 2059],
 [0, 2059, 540],
 [0, 1393, 2059],
 [0, 2059, 2117],
 [0, 2059, 647],
 [0, 2660, 1669],
 [0, 937, 180],
 [0, 729, 180],
 [0, 180, 540],
 [0, 1393, 180],
 [0, 937, 729],
 [0, 937, 540],
 [0, 937, 1393],
 [0, 937, 2117],
 [0, 937, 647],
 [0, 729, 540],
 [0, 729, 1393],
 [0, 729, 2117],
 [0, 729, 647],
 [0, 1393, 540],
 [0, 540, 2117],
 [0, 540, 647],
 [0, 1393, 2117],
 [0, 1393, 647],
 [0, 2117, 647],
 [2059, 2629, 1614],
 [180, 2629, 1614],
 [937, 2629, 1614],
 [729, 2629, 1614],
 [540, 2629, 1614],
 [1393, 2629, 1614],
 [2117, 2629, 1614],
 [2629, 1614, 647],
 [2059, 180, 1614],
 [937, 2059, 1614],
 [729, 2059

In [48]:
count_candidate3_df = count_support(df, candidate3_list)
freq_itemset3 = filter_sup(count_candidate3_df)
freq_itemset3

,itemset,sup
0,"[0, 2629, 1614]",63
1,"[0, 2059, 1614]",63
2,"[0, 1669, 1614]",63
3,"[0, 180, 1614]",62
4,"[0, 937, 1614]",61
...,...,...
147,"[729, 2117, 647]",51
148,"[1393, 540, 2117]",51
149,"[1393, 540, 647]",51
150,"[540, 2117, 647]",51


## All Frequent Itemset

In [49]:
frequent_itemset = pd.concat([freq_itemset1, freq_itemset2, freq_itemset3], axis=0)
frequent_itemset_final = frequent_itemset.reset_index(drop=True)
frequent_itemset_final

,itemset,sup
0,[0],1003
1,[232],427
2,[1614],127
3,[1536],113
4,[2629],105
...,...,...
233,"[729, 2117, 647]",51
234,"[1393, 540, 2117]",51
235,"[1393, 540, 647]",51
236,"[540, 2117, 647]",51


# Find Association Rules

In [50]:
def get_subset_list(candidate):
    temp = []
    final = []
    for i in range(len(candidate)):
        for j in range(len(candidate)):
            if i != j:
                temp.append(candidate[j])
        temp_set = list(set(temp))
        if temp_set not in final :
          final.append(temp_set)
        temp_set = [candidate[i]]
        if temp_set not in final :
          final.append(temp_set)
        temp.clear()
    # print('Subset from {} : {}'.format(candidate, final))
    return final

In [51]:
counter_rule = 0
for itemset in freq_itemset2.values : 
    list_candidate = itemset[0]
    temp_candidate = (get_subset_list(list_candidate))
    df_candidate=freq_itemset1.loc[freq_itemset1['itemset'].isin(temp_candidate)]
    min_support = min(df_candidate['sup'])
    min_support_itemset =df_candidate.loc[df_candidate['sup']==min_support]
    print("+++++++++++++++++++++++++++")
    result=list(filter(lambda v: v not in min_support_itemset['itemset'].values[0], list_candidate))
    print(str(min_support_itemset['itemset'].values[0][0])+" -> "+(str(result[0])))
    print(" confidence score : "+str(round(int(itemset[1])/min_support,3)))
    counter_rule+=1
for itemset in freq_itemset3.values : 
    list_candidate = itemset[0]
    temp_candidate = (get_subset_list(list_candidate))
    df_candidate=freq_itemset1.loc[freq_itemset1['itemset'].isin(temp_candidate)]
    df_candidate2=freq_itemset2.loc[freq_itemset2['itemset'].isin(temp_candidate)]
    min_support = min(df_candidate['sup'])
    min_support_itemset =df_candidate.loc[df_candidate['sup']==min_support]
    print("+++++++++++++++++++++++++++")
    result=list(filter(lambda v: v not in min_support_itemset['itemset'].values[0], list_candidate))
    # df_candidate = df_candidate.drop(min_support_itemset.index)
    print(str(min_support_itemset['itemset'].values[0][0])+" ->"+(str(result)))
    print(" confidence score : "+str(round(int(itemset[1])/min_support,3)))
    counter_rule+=1
print("______________________________")
print("count of all rule : "+str(counter_rule))
print("______________________________")   



+++++++++++++++++++++++++++
232 -> 0
 confidence score : 0.215
+++++++++++++++++++++++++++
1614 -> 0
 confidence score : 1.0
+++++++++++++++++++++++++++
1536 -> 0
 confidence score : 0.947
+++++++++++++++++++++++++++
2629 -> 0
 confidence score : 0.99
+++++++++++++++++++++++++++
1880 -> 0
 confidence score : 0.884
+++++++++++++++++++++++++++
1699 -> 0
 confidence score : 0.985
+++++++++++++++++++++++++++
2059 -> 0
 confidence score : 1.0
+++++++++++++++++++++++++++
1669 -> 0
 confidence score : 1.0
+++++++++++++++++++++++++++
131 -> 0
 confidence score : 1.0
+++++++++++++++++++++++++++
180 -> 0
 confidence score : 1.0
+++++++++++++++++++++++++++
937 -> 0
 confidence score : 1.0
+++++++++++++++++++++++++++
729 -> 0
 confidence score : 1.0
+++++++++++++++++++++++++++
540 -> 0
 confidence score : 1.0
+++++++++++++++++++++++++++
2096 -> 0
 confidence score : 1.0
+++++++++++++++++++++++++++
2660 -> 0
 confidence score : 1.0
+++++++++++++++++++++++++++
2058 -> 0
 confidence score : 1.0
+++++